In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../Data/Automobile_data.csv")  

In [ ]:
df.head(12).T 

In [ ]:
df.info() 
df.describe() 

Hay algunos datos que figuran como object en vez de como corresponden. Esto es porque no aparecen como NaN y aparecen como ?
Se prosede a arregrar remplazando por nan y luego para identificar valores nulos

In [ ]:
df.replace('?', np.nan, inplace = True) 
print(df.isnull().sum()) 

In [ ]:

#Luego de verificar el dataset se considera a estas columnas como numericas que estan como object
df['normalized-losses'] = pd.to_numeric(df['normalized-losses'], errors = 'coerce')
df['bore'] = pd.to_numeric(df['bore'], errors='coerce')
df['stroke'] = pd.to_numeric(df['stroke'], errors='coerce')

#normalized-losses lo completaré con el promedio
df['normalized-losses'] = df['normalized-losses'].fillna(df['normalized-losses'].mean())
#numero de puertas con moda
df['num-of-doors'] = df['num-of-doors'].fillna(df['num-of-doors'].mode()[0])
#bore y stroke también hare con el promedio

df['bore'] = df['bore'].fillna(df['bore'].mean())
df['stroke'] = df['stroke'].fillna(df['stroke'].mean())
#Horsepower, peak rpm y price considero que es muy importante entonces no lo promedio, mejor lo elimino
df.dropna(subset = ['horsepower', 'peak-rpm', 'price'], inplace = True)

print(df.isnull().sum())


In [ ]:
print(df.dtypes)

In [ ]:
#Modifiquemos los tipos de datos de las columnas que sean necesarias
df['normalized-losses'] = df['normalized-losses'].astype(int)
df['num-of-doors'] = df['num-of-doors'].astype('category')
df['horsepower'] = df['horsepower'].astype(int)
df['peak-rpm'] = df['peak-rpm'].astype(int)
print(df.dtypes)

In [ ]:
#Ahora trataremos los outliers
sns.boxplot(data = df[['horsepower', 'city-mpg', 'highway-mpg']])
plt.show()

In [ ]:
#Considero que estos valores son posibles así que no los elimino
#Ahora haremos un análisis de correlación entre las variables
df_numeric = df.select_dtypes(include = ['int64', 'float64'])
corr_matrix = df_numeric.corr(method = 'pearson') 
plt.figure(figsize = (12, 10))
sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm')
plt.show()

In [ ]:
results = {} 
#Ahora seleccionaremos las variables que vamos a usar
X = df.drop(['price'], axis = 1) 
y = df['price'].astype(float)

#Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Debemos dividir las columnas categóricas y numéricas
categorical_features = X.select_dtypes(include = ['object']).columns
numerical_features = X.select_dtypes(include=['number']).columns

#Creamos el preprocesador usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown = 'ignore'), categorical_features)
    ]
)
#Creamos el pipeline para la regresión lineal
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

#Ajustamos el modelo a los datos de entrenamiento
pipeline.fit(X_train, y_train)

#Realizamos las predicciones
y_pred = pipeline.predict(X_test)
#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['Regresión lineal'] = {'MSE': mse, 'R2': r2}
print(f'Regresión lineal - MSE: {mse:.2f}, R2: {r2:.2f}')

#Ahora vamos con el K-Nearest Neighbors
#Creamos el pipeline para el KNN
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors = 5))
])

#Entrenamos el modelo
pipeline.fit(X_train, y_train)

#Realizamos las predicciones
y_pred = pipeline.predict(X_test)

#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['KNN'] = {'MSE': mse, 'R2': r2}
print(f'KNN - MSE: {mse:.2f}, R2: {r2:.2f}')

#Ahora vamos el el árbol de decisión
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state = 0))
])

#Entrenamos el modelo
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

#Evaluamos el rendimiento (MSE y R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
results['Arbol de decisión'] = {'MSE': mse, 'R2': r2}
print(f'Arbol de decisión: - MSE: {mse:.2f}, R2: {r2:.2f}')

In [ ]:
#Ahora comparamos los resultados de los tres modelos
results_df = pd.DataFrame(results).T
print(results_df)

El análisis indica que el Árbol de Decisión es el modelo más adecuado, con el menor error (MSE) y un R² de 0.88, lo que demuestra su capacidad para explicar la mayoría de la variabilidad en los datos. En contraste, el KNN tiene un rendimiento moderado, y la Regresión Lineal presenta resultados deficientes. Para mejorar, se recomienda realizar validación cruzada, ajustar los hiperparámetros del Árbol de Decisión para evitar sobreajuste y probar modelos más avanzados como Random Forest o Gradient Boosting, optimizando así la precisión y robustez del modelo.